# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 31 2022 9:12AM,249678,10,Stone8714704,Acute Outpatient Solutions,Released
1,Oct 31 2022 9:10AM,249676,10,MSP215312,"Methapharm, Inc.",Released
2,Oct 31 2022 9:10AM,249676,10,MSP215313,"Methapharm, Inc.",Released
3,Oct 31 2022 9:10AM,249676,10,MSP215315,"Methapharm, Inc.",Released
4,Oct 31 2022 9:10AM,249676,10,MSP215317,"Methapharm, Inc.",Released
5,Oct 31 2022 9:10AM,249676,10,MSP215318,"Methapharm, Inc.",Released
6,Oct 31 2022 9:10AM,249676,10,MSP215319,"Methapharm, Inc.",Released
7,Oct 31 2022 9:10AM,249676,10,MSP215321,"Methapharm, Inc.",Released
8,Oct 31 2022 9:10AM,249676,10,MSP215322,"Methapharm, Inc.",Released
9,Oct 31 2022 9:10AM,249676,10,MSP215324,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,249673,Released,33
18,249674,Released,10
19,249675,Released,3
20,249676,Released,33
21,249678,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
249673,NaN,33.0
249674,NaN,10.0
249675,NaN,3.0
249676,NaN,33.0
249678,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
249646,6.0,3.0
249651,3.0,15.0
249654,0.0,7.0
249656,0.0,2.0
249657,0.0,56.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
249646,6,3
249651,3,15
249654,0,7
249656,0,2
249657,0,56


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,249646,6,3
1,249651,3,15
2,249654,0,7
3,249656,0,2
4,249657,0,56


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,249646,6,3
1,249651,3,15
2,249654,,7
3,249656,,2
4,249657,,56


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 31 2022 9:12AM,249678,10,Acute Outpatient Solutions
1,Oct 31 2022 9:10AM,249676,10,"Methapharm, Inc."
34,Oct 31 2022 9:09AM,249674,10,"Methapharm, Inc."
44,Oct 31 2022 9:08AM,249675,20,"Exact-Rx, Inc."
47,Oct 31 2022 9:07AM,249673,10,ISDIN Corporation
80,Oct 31 2022 9:04AM,249671,10,ISDIN Corporation
143,Oct 31 2022 9:04AM,249670,10,ISDIN Corporation
192,Oct 31 2022 9:00AM,249667,10,Bio-PRF
212,Oct 31 2022 8:59AM,249669,12,Hush Hush
215,Oct 31 2022 8:59AM,249668,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 31 2022 9:12AM,249678,10,Acute Outpatient Solutions,,1
1,Oct 31 2022 9:10AM,249676,10,"Methapharm, Inc.",,33
2,Oct 31 2022 9:09AM,249674,10,"Methapharm, Inc.",,10
3,Oct 31 2022 9:08AM,249675,20,"Exact-Rx, Inc.",,3
4,Oct 31 2022 9:07AM,249673,10,ISDIN Corporation,,33
5,Oct 31 2022 9:04AM,249671,10,ISDIN Corporation,,63
6,Oct 31 2022 9:04AM,249670,10,ISDIN Corporation,,49
7,Oct 31 2022 9:00AM,249667,10,Bio-PRF,7,13
8,Oct 31 2022 8:59AM,249669,12,Hush Hush,,3
9,Oct 31 2022 8:59AM,249668,12,Hush Hush,,36


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 9:12AM,249678,10,Acute Outpatient Solutions,1,
1,Oct 31 2022 9:10AM,249676,10,"Methapharm, Inc.",33,
2,Oct 31 2022 9:09AM,249674,10,"Methapharm, Inc.",10,
3,Oct 31 2022 9:08AM,249675,20,"Exact-Rx, Inc.",3,
4,Oct 31 2022 9:07AM,249673,10,ISDIN Corporation,33,
5,Oct 31 2022 9:04AM,249671,10,ISDIN Corporation,63,
6,Oct 31 2022 9:04AM,249670,10,ISDIN Corporation,49,
7,Oct 31 2022 9:00AM,249667,10,Bio-PRF,13,7
8,Oct 31 2022 8:59AM,249669,12,Hush Hush,3,
9,Oct 31 2022 8:59AM,249668,12,Hush Hush,36,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 9:12AM,249678,10,Acute Outpatient Solutions,1,
1,Oct 31 2022 9:10AM,249676,10,"Methapharm, Inc.",33,
2,Oct 31 2022 9:09AM,249674,10,"Methapharm, Inc.",10,
3,Oct 31 2022 9:08AM,249675,20,"Exact-Rx, Inc.",3,
4,Oct 31 2022 9:07AM,249673,10,ISDIN Corporation,33,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 9:12AM,249678,10,Acute Outpatient Solutions,1,NaN
1,Oct 31 2022 9:10AM,249676,10,"Methapharm, Inc.",33,NaN
2,Oct 31 2022 9:09AM,249674,10,"Methapharm, Inc.",10,NaN
3,Oct 31 2022 9:08AM,249675,20,"Exact-Rx, Inc.",3,NaN
4,Oct 31 2022 9:07AM,249673,10,ISDIN Corporation,33,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 9:12AM,249678,10,Acute Outpatient Solutions,1,0.0
1,Oct 31 2022 9:10AM,249676,10,"Methapharm, Inc.",33,0.0
2,Oct 31 2022 9:09AM,249674,10,"Methapharm, Inc.",10,0.0
3,Oct 31 2022 9:08AM,249675,20,"Exact-Rx, Inc.",3,0.0
4,Oct 31 2022 9:07AM,249673,10,ISDIN Corporation,33,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3744972,414,10.0
12,499337,39,0.0
19,249646,3,6.0
20,249675,3,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3744972,414,10.0
1,12,499337,39,0.0
2,19,249646,3,6.0
3,20,249675,3,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,414,10.0
1,12,39,0.0
2,19,3,6.0
3,20,3,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,414.0
1,12,Released,39.0
2,19,Released,3.0
3,20,Released,3.0
4,10,Executing,10.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20
Status,,,,
Executing,10.0,0.0,6.0,0.0
Released,414.0,39.0,3.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20
0,Executing,10.0,0.0,6.0,0.0
1,Released,414.0,39.0,3.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20
0,Executing,10.0,0.0,6.0,0.0
1,Released,414.0,39.0,3.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()